In [ ]:
import functools
import numpy as np
from scipy.linalg import expm

import qiskit as qk
import qiskit_dynamics as qk_d
import qiskit.providers.fake_provider as qk_fp

import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import importlib

# Setting up the Problem
-----

We would like to solve the following equation,
\begin{align}
i\frac{\partial}{\partial t}|\psi(t)\rangle=\hat{H}|\psi(t)\rangle
\end{align}
where, $\hat{H}$ describes the Hamiltonian of the Ising model,
\begin{align}
\hat{H}=\frac{1}{2}\sum_{i=0}^{N-2}J_x\sigma^x_i \sigma^x_{i+1}+\frac{1}{2}\sum_{i=0}^{N-1}h_{z}\sigma^z_{i}
\end{align}
This Hamiltonian describes a magnetic chain (or wire) consisting of $N$ magnetic atoms, where the magnetic atoms are arranged in a line. Each atom interacts with its nearest neighboring atom with the strength of the coupling given by $(J_x,J_y,J_z)$ in addition to an external magnetic field given by the vector $\vec{h}=(hx,hy,hz)$. For simplicity, consider the following values for the parameters,
\begin{align}
(J_x= 2\pi ) \textrm{ and } (hz=2\pi)
\end{align}

We also set the initial state to be have all spins in the $|+\rangle$ state,
\begin{align}
|\psi(t=0)\rangle&=|+\rangle^{\otimes N}\
&=\frac{1}{2^{N/2}}(|0\rangle+|1\rangle)\otimes(|0\rangle+|1\rangle)\otimes\cdots N\textrm{ times}
\end{align}



In particular, we will be interested in the time-dependence of the following quantity,
\begin{align}
\frac{1}{N}\left\langle \sum_{i=0}^{N-1} \sigma^z_i(t) \right\rangle=\frac{1}{N}\sum_i \langle\psi(t)| \sigma^z_i |\psi(t)\rangle
\end{align}

In [ ]:
# Set the parameters globally for the model
N = 4  # number of spins
hz = 1.0 * 2 * np.pi  # magnetic field along z
Jx = 1.0 * 2 * np.pi  # Coupling along x
Δt = 0.05  # time step for integration
tlist = np.arange(50) * Δt  # time values

# Global backend
backend = qk_fp.FakeManila()

Now we are ready to create the Unitary operator for the Ising Hamiltonian by composing one and two qubit gates. We show the first and second order Trotter decomposition where, for $\hat{H}=\hat{A}+\hat{B}$, 
\begin{align}
e^{-i\hat{H}\Delta t}&\approx e^{-i\hat{A}\Delta t}e^{-i\hat{B}\Delta t} + O(\Delta t^2)\textrm{ (First Order Trotter)}\
e^{-i\hat{H}\Delta t}&\approx e^{-i\hat{B}\frac{\Delta t}{2}} e^{-i\hat{A}\Delta t}e^{-i\hat{B}\frac{\Delta t}{2}}+O(\Delta t^3) \textrm{ (Second Order Trotter)}
\end{align}

In [ ]:
def create_initial_state(qr, cr):
    '''
    Creates the initial state for the experiment.
    Here we would like to prepare the |+>|+>|+>|+> state.
    '''
    circ = qk.QuantumCircuit(qr, cr)
    [circ.h(qr[i]) for i in range(qr.size)]
    return circ


def rxx(circ, θx, q1, q2):
    '''
    Implements the exp(-i theta/2 sx_1 sx_2) gate.
    '''
    circ.cx(q1, q2)
    circ.rx(θx, q1)
    circ.cx(q1, q2)
    return circ


def first_order_Trotter_unitary(circ, Δt, barrier=False):
    '''
    Applies a Unitary for a single time step U(Δt) using first-order trotter expansion to the input quantum circuit.
    input : circ is a quantum circuit
    output : circ-U(Δt)-
    '''
    # Layer A: coupling XX between n and n+1
    # Induce parallel CR gates
    if barrier:
        circ.barrier()
    for p in range(0, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])
    if barrier:
        circ.barrier()
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])

    #  Alt., a naive loop will apply XX rotations sequentially, failing to stack.
    # for p in range(0, circ.num_qubits - 1):
    #     circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])

    # Layer B: on-site Z
    for p in range(circ.num_qubits):    
        circ.rz(hz*Δt, circ.qubits[p])

    return circ


def second_order_Trotter_unitary(circ, Δt):
    '''
    Applies a Unitary for a single time step U(Δt) using second-order trotter expansion to the input quantum circuit.
    input : circ is a quantum circuit
    output : circ-U(Δt)-
    '''
    # Layer 2: on-site Z (half)
    for p in range(circ.num_qubits):
        circ.rz(hz * Δt / 2, circ.qubits[p])
 
    # Layer 1: coupling XX between n and n+1
    # Induce parallel structure (and symmetry)
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt / 2, circ.qubits[p], circ.qubits[p+1])
    # NOTE: Add barriers to separate the parallel section
    circ.barrier()
    for p in range(0, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p+1])
    circ.barrier()
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt / 2, circ.qubits[p], circ.qubits[p+1])

    # Layer 2: on-site Z (half)
    for p in range(circ.num_qubits):
        circ.rz(hz * Δt / 2, circ.qubits[p])
    return circ

In [ ]:
qr = qk.QuantumRegister(N)
cr = qk.ClassicalRegister(N)
circ = create_initial_state(qr, cr)
circ.barrier()
circ = first_order_Trotter_unitary(circ, Δt)
circ.barrier()
# circ.measure(qr, cr)
circ.draw('mpl')

# Transpiler & scheduler
-----

Make a pass to get gate times.

In [ ]:
# transp_circ = qk.transpiler.passes.RemoveBarriers()(circ)
basis_gates = ['rz','x','sx', 'ecr', 'id']
transp_circ = circ
transp_circ  = qk.transpiler.passes.RemoveBarriers()(transp_circ)
transp_circ = qk.transpile(
    transp_circ,
    backend,
    basis_gates=basis_gates,
    optimization_level=1
)
transp_circ  = qk.transpiler.passes.RemoveBarriers()(transp_circ)
# transp_circ.draw('mpl')

ns = 1e-9
instruction_durations = qk.transpiler.InstructionDurations(
    [('rz', None, 0 * ns, 's'),
    ('x', None, 30 * ns, 's'),
    ('id', None, 30 * ns, 's'),
    ('ecr', None, 150 * ns, 's'),
    ('sx', None, 30 * ns, 's'),
    ("reset", None, 1500 * ns, 's'),
    ("measure", None, 1500 * ns, 's')],
    dt = ns
)

# dd_sequence = [qk.circuit.library.XGate(), qk.circuit.library.XGate()]
pm = qk.transpiler.PassManager(
    [qk.transpiler.passes.ASAPScheduleAnalysis(instruction_durations)],
    #  qk.transpiler.passes.PadDynamicalDecoupling(instruction_durations, dd_sequence)]
)
pm_circ = pm.run(transp_circ)
pm_circ.duration = 1_000 # dt

qk.visualization.timeline.draw(
    pm_circ,
    # style=qk.visualization.timeline.IQXSimple()
)

Compare to the actual qiskit schedule.

In [ ]:
sched_circ = qk.schedule(qk.transpile(circ, backend), backend, method='alap')
# sched_circ.draw()
sched_circ.filter(channels=[qk.pulse.DriveChannel(i) for i in range(5)] + [qk.pulse.ControlChannel(i) for i in range(5)]).draw()

# Simulator
-----

- Gates are applied to qubit registers. 
- A qubit register is attached to a circuit. 
- Each gate has a pulse implementation which it applies to a qubit or pair of qubits using 
a qubit model(s).
- We construct models from qubit registers and attach pulses to register
channels. 
- Virtual Z's are applied to set the correct carrier offset as a separate
 operation.

In [ ]:
# configure jax to use 64 bit mode
import jax
jax.config.update("jax_enable_x64", True)

# tell JAX we are using CPU
jax.config.update('jax_platform_name', 'cpu')

# set default backend
qk_d.array.Array.set_default_backend('jax')
qk_d.array.Array.default_backend()

In [ ]:
import sys
sys.path.append("../")
import pulse_simulator as ps

In [ ]:
importlib.reload(ps)

## Qiskit model

In [ ]:
## The next cells are the output of:
# backend.configuration().hamiltonian

## Most parameters live in ...
# configuration = backend.configuration()
# properties = backend.properties()

\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}

Qubits are modeled as Duffing oscillators. In this case, the system includes higher energy states, i.e. not just |0> and |1>. The Pauli operators are generalized via the following set of transformations:

$(\mathbb{I}-\sigma_{i}^z)/2 \rightarrow O_i \equiv b^\dagger_{i} b_{i}$

$\sigma_{+} \rightarrow b^\dagger$,

$\sigma_{-} \rightarrow b$

$\sigma_{i}^X \rightarrow b^\dagger_{i} + b_{i}$.

Qubits are coupled through resonator buses. The provided Hamiltonian has been projected into the zero excitation subspace of the resonator buses leading to an effective qubit-qubit flip-flop interaction. The qubit resonance frequencies in the Hamiltonian are the cavity dressed frequencies and not exactly what is returned by the backend defaults, which also includes the dressing due to the qubit-qubit interactions.

Quantities are returned in angular frequencies, with units $2\pi$ GHz.

WARNING: Currently not all system Hamiltonian information is available to the public, missing values have been replaced with 0.

## Device set up

__Comment on backends, circuits and registers (design choice):__
We want to simulate a circuit in a minimal way. Crosstalk prevents us from 
cutting up the circuit because everything is connected.

To handle qubit containers, there is both the backend and the circuit. I am
using the register to hold qubits that should be simulated. I use the backend
for parameters and anything else. I think the circuit should not be involved
in making the device model.

NOTE: Once a circuit is transpiled to a backend, the circuit qubits and the
backend qubits should be the same thing. There should be no difference in one
container over the other.

__Comment on variables:__
I am using the backend Hamiltonian to provide the variables of the simulator.
There are a few modifications that I am making. I also want the ability to
edit this dictionary manually, so I do not use the backend to get variables
when making Hamiltonians for the pulse simulator.

In [ ]:
# Initialize device
# =====
# Undo units
units = 1e9
GHz = 1/units
ns = units

dt = backend.configuration().dt * ns  
duration = 220 * dt  # ns

registers = [0, 1, 2, 3]  # TODO: Active registers

# Variables
# NOTE: If the Rabi rates are different, you have to calibrate!
config_vars = ps.backend_simulation_vars(backend, rabi=False, units=units)

# Carrier frequencies of each control line
carriers = ps.backend_carriers(backend, config_vars)

config_vars

Here is a demo of crosstalk.

In [ ]:
for edge in ps.backend_edges(backend):
    print(f"{edge}: {ps.zz_coupling(edge, config_vars)}")

In [ ]:
g = ps.backend_edges(backend)
print(g)

fig, ax = plt.subplots(figsize=[2,2])
ax.imshow(np.abs(ps.crosstalk_model([1, 2], g, config_vars)))

The next cell defines a toy pulse.


NOTE:   Careful with steps vs. durations in qiskit pulses! 
        They must be integers or you get an UnassignedDurationError.

This $f(x)$ is the "Gaussian" that Qiskit uses?
\begin{align}
    g(x) &= \exp\Bigl( -\frac12 \frac{{(x - \text{duration}/2)}^2}{\text{sigma}^2} \Bigr)\\
    f(x) &= \text{amp} \times \frac{g(x) - g(-1)}{1-g(-1)}, \quad 0 \le x < \text{duration}
\end{align}

https://qiskit.org/documentation/_modules/qiskit/pulse/library/parametric_pulses.html#Gaussian
https://qiskit.org/documentation/_modules/qiskit/pulse/library/symbolic_pulses.html

In [ ]:
def gaussian(x, mu=0, sigma=1):
    return np.exp(-(x - mu)**2 / 2 / sigma**2)


def lifted_gaussian(x, mu, sigma):
    g = functools.partial(gaussian, mu=mu, sigma=sigma)
    return (g(x) - g(-1)) / (1 - g(-1))


def gaussian_envelope(dt, duration, angle=np.pi):
    """ Define gaussian envelope function to accumulate the angle.

    TODO: Rabi?

    Returns:
        Qiskit pulse implementing angle.
    """
    steps = int(duration / dt)

    # Arbitrary shape
    # NOTE: Qiskit doesn't like pulse amplitudes > 1. Widen to avoid this.
    sigma = steps / 4
    mu = steps / 2

    # Integral
    x = np.linspace(0, steps, endpoint=True)
    area = np.trapz(lifted_gaussian(x, mu, sigma), x)

    # I have no idea what is going on here (amplitude = angle / area ?)
    mystery_factory = 0.6982
    amplitude = angle / area * (np.pi / mystery_factory)

    # Adjust the amplitude to achieve the angle
    return qk.pulse.Gaussian(steps, amplitude, sigma)

### Checking the confusing calibration

In [ ]:
g = gaussian_envelope(dt, duration, angle=np.pi)
g.draw()

In [ ]:
wave = g.get_waveform().samples.real
np.trapz(wave, dx=dt)

Trying to find an equivalent pulse.

In [ ]:
steps = int(duration / dt)
mu = duration / 2
sigma = duration / 5

x = np.linspace(0, duration, steps)
area = np.trapz(lifted_gaussian(x, mu=mu, sigma=sigma), x)
amplitude = np.pi / area

# Check
y = lifted_gaussian(x, mu=mu, sigma=sigma)
np.trapz(amplitude * y, dx=dt)

No idea how to compare the plots...

In [ ]:
fig, ax = plt.subplots()
ax.plot(x / dt, wave, label="Qiskit")
ax.plot(x / dt, y * dt, label="Numpy")
ax.legend()

## Single qubit moment

There are two separate functionalities: models and pulses.

### Pulses
Pulses will need to be define for each moment of the circuit. They will depend
on the gate

Our simulation is not going to be continuous across moments. I am not sure that I want to be keeping track of the carry for the virtual Z gates. Plus, I am not sure how to handle the virtual Z for the two qubit gates when the CR interaction is not block diagonal. (In particular, if we use a Hamiltonian for the CR drive that includes XI, which phase does XI pick up? The control line phase because the X operator acts on the control qubit? Or the target line phase because the cross-resonance drive is being carried by the target line.)

To get out from under this, we will just apply Z gates directly without any pulse synthesis and pretend we are handling virtual Z's.

In [ ]:
# Choose gates from the circuit
# ======
"""
NOTE:   Virtual Zs are attached to each qubit line. These are not quite 
        how they'd be implemented in software?
"""
virtual_zs = {
    0: np.pi / 2,
    1: 0.,
    2: np.pi / 2,
    3: 0.
}

gates = {
    0: "sx_red",
    1: "sx_red",
    2: "sx_red",
    3: "sx_red"
}

# Lookup table
# =====
# should also include "sx_blue", "x_blue"
gate_lookup = {
    "sx_red": gaussian_envelope(dt, duration, angle=np.pi/4),
    "x_red": gaussian_envelope(dt, duration, angle=np.pi/2)
}

# Design pulse schedule
# =====
with qk.pulse.build(name="Current moment") as pulse_moment:
    for i, gate in gates.items():
        channel = qk.pulse.DriveChannel(i)
        # NOTE:     Shift phase will adjust the carry and future gates. 
        #           Restricted to the current moment, it should be the same as 
        #           a zero time R_Z gate.
        qk.pulse.shift_phase(virtual_zs[i], channel)
        qk.pulse.play(gate_lookup[gate], channel)

### Models

We operate in the frame rotating with each qubit, and we assume that we induce dynamics which are given by an effective model. This effective model has an always-on $ZZ$ coupling,
\begin{equation}
    J^2 \left(\frac{1}{\Delta + \alpha}- \frac{1}{\Delta - \alpha} \right) ZZ
\end{equation}

In [ ]:
# Define models fixed by the current parameters
# =====
H_rx = functools.partial(
    ps.rx_model, 
    registers=registers,
    backend=backend,
    variables=config_vars,
    rotating_frame=True,
)

A_decay = functools.partial(
    ps.qubit_decay_model,
    registers=registers, 
    variables=config_vars
)


H_xtalk = ps.crosstalk_model(
    registers,
    ps.backend_edges(backend),
    config_vars
)

In [ ]:
# Create a system model
# =====
# Control model
#   Single qubit circuit moment: attach RX models to each qubit
H_drift = 0.
Hs_control = []
Hs_channels = []
As_static = []
for qubit, label in gates.items():
    
    # if qubit_index in registers:
    Hj_drift, Hjs_control, Hjs_channel = H_rx(qubit)

    # NOTE: doesn't work with quantum channel simulator yet
    Ajs_static = A_decay(qubit)

    H_drift += Hj_drift
    Hs_control += Hjs_control
    Hs_channels += Hjs_channel
    As_static += Ajs_static

# Construct the solver
# NOTE: no rotating frames for now
# =====
solver = qk_d.Solver(
    static_hamiltonian=H_xtalk,
    hamiltonian_operators=Hs_control,
    static_dissipators=None,
    rotating_frame=None,
    rwa_cutoff_freq=None,
    hamiltonian_channels=Hs_channels,
    channel_carrier_freqs={ch: 0. for ch in Hs_channels},
    dt=dt
)

# solver = qk_d.Solver(
#     static_hamiltonian=H_drift, # + H_xtalk,
#     hamiltonian_operators=Hs_control,
#     static_dissipators=None, # As_static if len(As_static) > 0 else None,
#     rotating_frame=H_drift,
#     rwa_cutoff_freq=2 * max(carriers.values()),
#     hamiltonian_channels=Hs_channels,
#     channel_carrier_freqs=carriers,
#     dt=dt
# )

### Solve the model + pulse

Solve three ways
1. State vector sim (sparse)
2. Unitary sim
3. Channel sim (not working)

- The job of constructing and running this circuit moment is to give you a final unitary at the end.
- If there is dissipation, you will need a final channel. This requires a superoperator representation of the initial unitiary. The channel can then be returned. This turns out to be very slow. How can I fix it?
- We will combine these final unitaries or channels into a list of moments. We will combine those moments to achieve a simulation. This should be possible in Qiskit using the evolve functionality, even if we have different representations for each moment (channel, unitary).

In [ ]:
# Start the qubit in its ground state.
y0 = qk.quantum_info.states.Statevector(
    functools.reduce(
        np.kron, 
        np.repeat([[1, 0]], len(registers), axis=0)
    )
)

# Identity matrix
id_label = ''.join(['I'] * len(registers))
U0 = qk.quantum_info.Operator.from_label(id_label)

# NOTE: Unsure if correct
# Identity channel
C0 = qk.quantum_info.SuperOp(U0)

In [ ]:
# Unitary or channel sim.
# NOTE: Bug when solving vectorized problems
if False: #len(As_static) > 0:
    solver.model.evaluation_mode = 'sparse_vectorized'
    sol = solver.solve(
        t_span=[0., duration], 
        y0=C0, 
        signals=pulse_moment,
        atol=1e-8,
        rtol=1e-8,
        method='jax_odeint'
    )

else:
    solver.model.evaluation_mode = 'sparse'
    sol = solver.solve(
        t_span=[0., duration],
        y0=U0, 
        signals=pulse_moment,
        atol=1e-8, 
        rtol=1e-8,
        method='jax_odeint'
    )

# Sparse state vector sim
solver.model.evaluation_mode = 'sparse'

sol1 = solver.solve(
    t_span=[0., duration],
    y0=y0,
    signals=pulse_moment,
    atol=1e-8,
    rtol=1e-8,
    method='jax_odeint'
)


In [ ]:
basis = ps.hilbert_space_basis([2] * len(transp_circ.qubits))

# Check final states
yf1 = sol1.y[-1]
y_after_1qb_moment = yf1

Uf = sol.y[-1]
yf = y0.evolve(Uf)

# Compare
print(f"Are equal? {yf1 == yf}\n")
print(f"Are close? ||y1 - y2|| = {np.linalg.norm(yf1 - yf)}\n")

# States
ps.print_wavefunction(yf, basis)
print()

# ps.print_wavefunction(yf1, basis)

In [ ]:
fig, ax = plt.subplots(figsize=[2,2])
ax.axis('off')
ax.imshow(np.abs(Uf))

### Aside: Plot pulses

In [ ]:
"""
- Signals are ordered by the channels of the pulse schedule.
- From the documentation: "Instances of ScheduleBlock must first be converted 
to Schedule using the block_to_schedule() function in Qiskit Pulse." This can
be accomplished with `qk.pulse.transforms.block_to_schedule(moment)`.

I didn't see that begin necessary currently.
""";

converter = qk_d.pulse.InstructionToSignals(dt, carriers=carriers)
signals = converter.get_signals(pulse_moment)

# TODO: probably a better way to set duration

fig, ax = ps.plot_pulse_schedule(
    signals, 
    pulse_moment, 
    duration=pulse_moment.duration / 4,
    figsize=[8, 6]
)
fig.tight_layout()

## Two qubit moment

__Unnamed model:__
Coupling in the lab frame:
\begin{equation}
    H^\text{lab}_J = J \left( (\sigma^{(1)}_+ + \sigma^{(1)}_-)(\sigma^{(2)}_+ + \sigma^{(2)}_-)\right) = J X X
\end{equation}

Making the rotating wave approximation (even without changing frames):
\begin{equation}
    H_J = J\left(\sigma^{(1)}_+ \sigma^{(2)}_- + \sigma^{(1)}_- \sigma^{(2)}_+ \right) = \frac{J}{2}(XX + YY)
\end{equation}

We can use this coupling to dress the qubit energies. This means changing to the
eigenbasis of the static Hamiltonian, so
\begin{equation}
    H_S = H_Q + H_J \rightarrow H_S W = W D \rightarrow H_S = W D W^\dag
\end{equation}

Then, we can make the same frame change to the Hamiltonian of the drive on the control
\begin{equation} 
    \Omega_T(t) XI \rightarrow  \Omega_T(t) W^\dag XI W
\end{equation}

... missing step...

\begin{equation}
    H_D(t) = \Omega_T(t) \left( XI - \frac{J}{\Delta_{12}} ZX \right)
\end{equation}

-----

__SWPT model:__
The drives, to first order (assuming adiabatic drives),
\begin{equation}
   \frac{1}{2}\left(\frac{1}{2(\Delta + \alpha)}- \frac{1}{2 \Delta} \right) \Omega^2_C(t) ZI + \left(\Omega_T(t) + \frac{J}{\Delta + \alpha} \Omega_C(t) \right) IX + \left(\frac{J}{\Delta + \alpha} - \frac{J}{\Delta} \right) \Omega_C(t) ZX
\end{equation}
or
\begin{equation}
   \frac{1}{2}\left(\frac{1}{2(\Delta + \alpha)} - \frac{1}{2 \Delta} \right) \Omega^2_C(t) ZI + \Omega_T(t) IX + \frac{J}{\Delta + \alpha} \left( IX - \frac{\alpha}{\Delta}ZX \right)  \Omega_C(t) 
\end{equation}

We don't care about the phase coming from ZI. Then, 
\begin{equation}
    H_\text{eff}(t) = \Omega_T(t) IX + \frac{J}{\Delta + \alpha} \left( IX - \frac{\alpha}{\Delta}ZX \right)  \Omega_C(t) 
\end{equation}

The effect of virtual Z's should simply change the action of the control drive to be $\cos(\phi)IX + \sin(\phi)IX$ and $\cos(\phi)ZX + \sin(\phi)ZY$, and this can be accomplished with shift phase like for the $X$ gate.



In [ ]:
# Choose gates
# ======
virtual_zs = {
    0: 0,
    1: np.pi / 2,
    2: 0.,
    3: np.pi / 2
}

gates = {
    (0,1): "zx_red",
    (2,3): "zx_red",
}

control_gate_lookup = {
    "zx_red": gaussian_envelope(dt, duration, angle=np.pi/2),
}
target_gate_lookup = {
    "zx_red": gaussian_envelope(dt, duration, angle=np.pi/2),
}

# Design pulse schedule
# =====
with qk.pulse.build(name="Two moment") as two_pulse_moment:
    for (c, t), gate in gates.items():
        control_channel = ps.get_control_channel(c, t, backend)
        target_channel = ps.get_drive_channel(t, backend)

        # Probably replace Virtual Z => Zero time Rz gate
        for i in virtual_zs.items():
            if i == c:
                qk.pulse.shift_phase(virtual_zs[i], control_channel)
            elif i == t:
                qk.pulse.shift_phase(virtual_zs[i], target_channel)

        # Pulses must work together
        qk.pulse.play(control_gate_lookup[gate], control_channel)
        qk.pulse.play(target_gate_lookup[gate], target_channel)

In [ ]:
# Create a system model
# =====

# Partially compile to get this circuit's gates
cr_model = functools.partial(
    ps.cross_resonance_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
)


# Control model
H_drift = 0.
Hs_control = []
Hs_channels = []
for (control, target), label in gates.items():
    Hj_drift, Hjs_control, Hjs_channel = cr_model((control, target))
    H_drift += Hj_drift
    Hs_control += Hjs_control
    Hs_channels += Hjs_channel
    
# Construct the solver
# =====
"""
Simulating the effective model, therefore the drift
is ZZ crosstalk, and there is no rotating frame.

Use the crosstalk computed perviously.
"""
solver = qk_d.Solver(
    static_hamiltonian=None, #H_xtalk,
    hamiltonian_operators=Hs_control,
    static_dissipators=None,
    rotating_frame=None,
    rwa_cutoff_freq=None,
    hamiltonian_channels=Hs_channels,
    channel_carrier_freqs={ch: 0. for ch in Hs_channels},
    dt=dt
)

In [ ]:
# Start the qubit in its ground state.
y0 = qk.quantum_info.states.Statevector(
    functools.reduce(
        np.kron, 
        np.repeat([[1, 0]], len(registers), axis=0)
    )
)

# # Start from previous moment
# y0 = y_after_1qb_moment

# Identity matrix
id_label = ''.join(['I'] * len(registers))
U0 = qk.quantum_info.Operator.from_label(id_label)

# NOTE: Unsure if correct
# Identity channel
C0 = qk.quantum_info.SuperOp(U0)

In [ ]:
# Unitary or channel sim.

# NOTE: Bug when solving vectorized problems
if False: #len(As_static) > 0:
    solver.model.evaluation_mode = 'sparse_vectorized'
    sol = solver.solve(
        t_span=[0., duration], 
        y0=C0, 
        signals=pulse_moment,
        atol=1e-8,
        rtol=1e-8,
        method='jax_odeint'
    )

else:
    solver.model.evaluation_mode = 'sparse'
    sol = solver.solve(
        t_span=[0., duration],
        y0=U0, 
        signals=pulse_moment,
        atol=1e-8, 
        rtol=1e-8,
        method='jax_odeint'
    )

# Sparse state vector sim
solver.model.evaluation_mode = 'sparse'

sol1 = solver.solve(
    t_span=[0., duration],
    y0=y0,
    signals=pulse_moment,
    atol=1e-8,
    rtol=1e-8,
    method='jax_odeint'
)

In [ ]:
basis = ps.hilbert_space_basis([2] * len(transp_circ.qubits))

# Check final states
yf1 = sol1.y[-1]

Uf = sol.y[-1]
yf = y0.evolve(Uf)

# Compare
print(f"Are close? ||y1 - y2|| = {np.linalg.norm(yf1 - yf)}\n")

# States
ps.print_wavefunction(yf, basis)
print()

Reasonable operators?



In [ ]:
fig, ax = plt.subplots(figsize=[2,2])
ax.axis('off')
ax.imshow(np.abs(Uf))

fig, ax = plt.subplots(figsize=[2,2])
ax.axis('off')
ax.imshow(np.sum([
    np.abs(expm(4 * Hs_control[i]))
    for i in range(4)], axis=0)
)